In [2]:
import os
import pickle
import numpy as np
import pandas as pd


In [3]:
!ls ../../data/raw

00_top_wikipedia_articles.zip
01_labeled_data.zip
02_expanded_labeled_data.zip
03_expanded_labeled_filtered.zip
04_expanded_labeled_filtered_parsed_titles.zip
05_expanded_labaled_removed_comparisons.zip
06_unique_articles_visited_api.csv
07_updated_unique_articles_visited_api.csv
08_redirect_info.pkl
raw_pickled


In [4]:
api_info = os.listdir("../../data/raw/raw_pickled/pickled_article_info")

In [ ]:
unpickled = []
for file in api_info:
    try:
        with open(f"../../data/raw/raw_pickled/pickled_article_info/{file}", "rb") as f:
            unpickled.append(pickle.load(f))
    except:
        continue
        

In [6]:
article_info = pd.concat(unpickled).reset_index().drop("index", axis=1)

In [7]:
article_info.shape

(111521, 5)

In [8]:
def extract_na(extract):
    if len(extract) == 0:
        return np.NaN
    return " ".join(extract)

article_info.extract = article_info.extract.apply(extract_na)

In [10]:
article_info.dropna(inplace=True)

In [15]:
article_info.head()

,title,extract,links,linkshere,categories
0,Q-carbon,Q-carbon (quenched carbon) is an allotrope of ...,"[Activated carbon, Aggregated diamond nanorod,...","[Allotropy, Carbon, Carbon nanotube, Diamond, ...","[Category:2015 in science, Category:Carbon forms]"
1,Optical_engineering,Optical engineering is the field of study that...,"[Acoustical engineering, Aerospace engineering...","[Minute and second of arc, Biomedical engineer...","[Category:Engineering disciplines, Category:En..."
2,Ray_transfer_matrix_analysis,Ray transfer matrix analysis (also known as AB...,"[Accelerator physics, Anthony E. Siegman, Baha...","[Lens (optics), Gaussian beam, Index of electr...","[Category:Accelerator physics, Category:Geomet..."
3,Photographic_lens_design,The design of photographic lenses for use in s...,"[.design, AIGA, Abbe number, Activity-centered...","[Processor design, Design, Design pattern, Fas...","[Category:Industrial design, Category:Photogra..."
4,Stray_light,"Stray light is light in an optical system, whi...","[ASAP (software), Anti-reflective coating, Bib...","[Light pollution, Diffraction grating, Ultravi...","[Category:Optical metrology, Category:Photometry]"


# Merge Redirect

In [16]:
with open("../../data/raw/08_redirect_info.pkl", "rb") as f:
    redirects = pickle.load(f)

In [43]:
redirects.head()

,title,redirects
0,Icosian,"[Icosians, Icosian group, Icosion]"
1,F4_(mathematics),"[F4 (math), F₄, F4 mathematics]"
2,E8_lattice,"[E8 root lattice, Gosset lattice, Gosset tesse..."
3,Mental_calculation#nat-exp,"[Techniques of Mental Calculation, Mental Calc..."
4,Carlitz_exponential,[]


In [20]:
info_redirects = article_info.merge(redirects, on="title", how="inner")

In [21]:
info_redirects.head()

,title,extract,links,linkshere,categories,redirects
0,Q-carbon,Q-carbon (quenched carbon) is an allotrope of ...,"[Activated carbon, Aggregated diamond nanorod,...","[Allotropy, Carbon, Carbon nanotube, Diamond, ...","[Category:2015 in science, Category:Carbon forms]",[Q-Carbon]
1,Optical_engineering,Optical engineering is the field of study that...,"[Acoustical engineering, Aerospace engineering...","[Minute and second of arc, Biomedical engineer...","[Category:Engineering disciplines, Category:En...","[Optical technology, Optical engineer]"
2,Ray_transfer_matrix_analysis,Ray transfer matrix analysis (also known as AB...,"[Accelerator physics, Anthony E. Siegman, Baha...","[Lens (optics), Gaussian beam, Index of electr...","[Category:Accelerator physics, Category:Geomet...","[Ray transfer matrix, ABCD matrix analysis, AB..."
3,Photographic_lens_design,The design of photographic lenses for use in s...,"[.design, AIGA, Abbe number, Activity-centered...","[Processor design, Design, Design pattern, Fas...","[Category:Industrial design, Category:Photogra...","[Camera lens design, Photographic lens constru..."
4,Stray_light,"Stray light is light in an optical system, whi...","[ASAP (software), Anti-reflective coating, Bib...","[Light pollution, Diffraction grating, Ultravi...","[Category:Optical metrology, Category:Photometry]",[]


add redirects to linkshere and then remove redirects

In [23]:
def combine_linkshere(row):
    row.linkshere += row.redirects
    return pd.Series(row)
        
combined_linkshere_redirects = info_redirects.apply(combine_linkshere, axis=1).drop('redirects', axis=1)

remove any duplicate in linkshere

In [28]:
combined_linkshere_redirects.linkshere = combined_linkshere_redirects.linkshere.apply(lambda x: list(set(x)))

In [30]:
combined_linkshere_redirects.head()

,title,extract,links,linkshere,categories
0,Q-carbon,Q-carbon (quenched carbon) is an allotrope of ...,"[Activated carbon, Aggregated diamond nanorod,...","[Diamond, Graphene, Allotropy, Carbon nanocone...","[Category:2015 in science, Category:Carbon forms]"
1,Optical_engineering,Optical engineering is the field of study that...,"[Acoustical engineering, Aerospace engineering...","[Praveen Kumar Gorakavi, Engineer, Charles Mun...","[Category:Engineering disciplines, Category:En..."
2,Ray_transfer_matrix_analysis,Ray transfer matrix analysis (also known as AB...,"[Accelerator physics, Anthony E. Siegman, Baha...","[Ray (optics), List of laser articles, Matrix ...","[Category:Accelerator physics, Category:Geomet..."
3,Photographic_lens_design,The design of photographic lenses for use in s...,"[.design, AIGA, Abbe number, Activity-centered...","[Design for manufacturability, Design quality ...","[Category:Industrial design, Category:Photogra..."
4,Stray_light,"Stray light is light in an optical system, whi...","[ASAP (software), Anti-reflective coating, Bib...","[Near-field scanning optical microscope, Coron...","[Category:Optical metrology, Category:Photometry]"


# Original Comparisons

In [13]:
comparisons = pd.read_csv("../../data/raw/05_expanded_labaled_removed_comparisons.zip")

In [14]:
comparisons.head()

,parent_title,parent_url,child_title,child_url,comparison
0,Division_by_Zero_(story),https://en.wikipedia.org/wiki/Division_by_Zero...,Division_by_zero,https://en.wikipedia.org/wiki/Division_by_zero,Division_by_Zero_(story)|Division_by_zero
1,Division_by_Zero_(story),https://en.wikipedia.org/wiki/Division_by_Zero...,Infinity,https://en.wikipedia.org/wiki/Infinity,Division_by_Zero_(story)|Infinity
2,Division_by_Zero_(story),https://en.wikipedia.org/wiki/Division_by_Zero...,Zero,https://en.wikipedia.org/wiki/Zero,Division_by_Zero_(story)|Zero
3,Indeterminate_form,https://en.wikipedia.org/wiki/Indeterminate_form,Defined_and_undefined,https://en.wikipedia.org/wiki/Defined_and_unde...,Defined_and_undefined|Indeterminate_form
4,Indeterminate_form,https://en.wikipedia.org/wiki/Indeterminate_form,Division_by_zero,https://en.wikipedia.org/wiki/Division_by_zero,Division_by_zero|Indeterminate_form


merge article info for parent

In [44]:
comparisons.rename(columns={"parent_title": "title"}, inplace=True)

In [45]:
merged_parent = comparisons.merge(
    combined_linkshere_redirects, on="title", how="inner")

merged_parent = merged_parent.merge(redirects, on="title", how="left")

In [47]:
merged_parent.rename(columns={"title": "parent_title",
                              "extract": "parent_extract", 
                              "links": "parent_links", 
                              "linkshere": "parent_linkshere", 
                              "categories": "parent_categories",
                              "redirects": "parent_redirects"},
                    inplace=True)

merge article info with child

In [49]:
merged_parent.rename(columns={"child_title": "title"}, inplace=True)

In [50]:
full_merged = merged_parent.merge(combined_linkshere_redirects, on="title", how="inner")

full_merged = full_merged.merge(redirects, on="title", how="left")

In [52]:
full_merged.rename(columns={"title": "child_title",
                              "extract": "child_extract", 
                              "links": "child_links", 
                              "linkshere": "child_linkshere", 
                              "categories": "child_categories",
                              "redirects": "child_redirects"}, inplace=True)

In [53]:
full_merged.head()

,parent_title,parent_url,child_title,child_url,comparison,parent_extract,parent_links,parent_linkshere,parent_categories,parent_redirects,child_extract,child_links,child_linkshere,child_categories,child_redirects
0,Division_by_Zero_(story),https://en.wikipedia.org/wiki/Division_by_Zero...,Division_by_zero,https://en.wikipedia.org/wiki/Division_by_zero,Division_by_Zero_(story)|Division_by_zero,Division by Zero is a science fiction short st...,"[Amazon.com, Anthology, Bantam Spectra, Consis...","[Division by zero (disambiguation), Stories of...","[Category:1991 short stories, Category:Science...",[],"In mathematics, division by zero is division w...","[0 (number), Alfred Tarski, Algebra, Android (...","[Divide by 0 Error, Division By Zero, Extended...","[Category:0 (number), Category:Computer arithm...","[Divide by zero, One over Zero, One Over Zero,..."
1,Indeterminate_form,https://en.wikipedia.org/wiki/Indeterminate_form,Division_by_zero,https://en.wikipedia.org/wiki/Division_by_zero,Division_by_zero|Indeterminate_form,In calculus and other branches of mathematical...,"[Analytic function, Calculus, Cardinal number,...","[Exponentiation, Indeterminate expression, L'H...",[Category:Limits (mathematics)],"[0/0, Zero divided by zero, 0 divided by 0, In...","In mathematics, division by zero is division w...","[0 (number), Alfred Tarski, Algebra, Android (...","[Divide by 0 Error, Division By Zero, Extended...","[Category:0 (number), Category:Computer arithm...","[Divide by zero, One over Zero, One Over Zero,..."
2,0,https://en.wikipedia.org/wiki/0,Division_by_zero,https://en.wikipedia.org/wiki/Division_by_zero,0|Division_by_zero,0 (zero) is both a number and the numerical di...,"[-1 (number), 0 (disambiguation), 0 (number), ...","[7000 (number), 19 (number), 178 (number), EBC...","[Category:0 (number), Category:Elementary arit...","[Number 0, Nought, Zeroes, Zero (mathematics),...","In mathematics, division by zero is division w...","[0 (number), Alfred Tarski, Algebra, Android (...","[Divide by 0 Error, Division By Zero, Extended...","[Category:0 (number), Category:Computer arithm...","[Divide by zero, One over Zero, One Over Zero,..."
3,Division_by_Zero_(story),https://en.wikipedia.org/wiki/Division_by_Zero...,Infinity,https://en.wikipedia.org/wiki/Infinity,Division_by_Zero_(story)|Infinity,Division by Zero is a science fiction short st...,"[Amazon.com, Anthology, Bantam Spectra, Consis...","[Division by zero (disambiguation), Stories of...","[Category:1991 short stories, Category:Science...",[],Infinity (symbol: \n \n \n \n ...,"[0.999..., 1,000,000, 1,000,000,000, 10,000, 1...","[Iterated binary operation, Bill Childers, Chr...","[Category:CS1: long volume value, Category:Con...","[The Infinite, Infinitely, Infinate, InFINity,..."
4,Indeterminate_form,https://en.wikipedia.org/wiki/Indeterminate_form,Extended_real_number_line,https://en.wikipedia.org/wiki/Extended_real_nu...,Extended_real_number_line|Indeterminate_form,In calculus and other branches of mathematical...,"[Analytic function, Calculus, Cardinal number,...","[Exponentiation, Indeterminate expression, L'H...",[Category:Limits (mathematics)],"[0/0, Zero divided by zero, 0 divided by 0, In...","In mathematics, the affinely extended real num...","[Absolute value, Algebraic number, Arithmetica...","[Number line, L'Hôpital's rule, 1,000,000,000,...","[Category:Infinity, Category:Real numbers]","[Extended real number, Extended reals, Extende..."


# Save Full Merged Dataframe

In [54]:
with open("../../data/interim/01_article_comparison_info.pkl", "wb") as f:
    pickle.dump(full_merged, f, 3)